In [1]:
import numpy as np
from scipy import stats

In [2]:
def Get_OBJ_Data(path_to_data, file_name):
    # Dictionary storing OBJ data
    obj = {
    "v": [],
    "f": []
    }

    # Parses and reads OBJ
    # Assumes Specific Format
    with open(f"{path_to_data}{file_name}.obj") as f:
        for i, line in enumerate(f):
            tokenized_line = line.split()
            key = tokenized_line[0]
            
            if key in ["mtllib", "usemtl"]:
                # could be important in the future             
                continue
            elif key == "f":
                numbers_pairs = [x.split("/") for x in tokenized_line[1:]]
                obj[key].append(list(map(lambda x: int(x[0]), numbers_pairs)))
            else:
                numbers = [float(x) for x in tokenized_line[1:]]
                obj[key].append(numbers)

    for key,value in obj.items():
        obj[key] = np.array(value)

    return obj

In [3]:
obj = Get_OBJ_Data('objs/p3d/results/', 'mesh_1_cube')
f = np.round(obj['v'][obj['f'][0] - 1] * 49)
v1 = f[1] - f[0]
v2 = f[2] - f[0]

norm = np.cross(v1, v2)
print(f)

midpoint = (f[1] - f[2]) /2 + f[2] - norm / 2


print(midpoint)

[[ 0.  2. 36.]
 [ 0.  2. 37.]
 [ 0.  3. 36.]]
[ 0.5  2.5 36.5]


In [4]:
cube_world = np.load('objs/p3d/results/cube_world_mesh_1.npy')
faces = np.round(obj['v'][(obj['f'] - 1)[:]] * 49).astype('int32')
v1 = faces[:, 1] - faces[:, 0]
v2 = faces[:, 2] - faces[:, 0]

print(v1.shape)
print(v2.shape)

norm = np.cross(v1, v2)
print(norm.shape)
print(faces.shape)

midpoint = (faces[:, 1] - faces[:, 2]) /2 + faces[:, 2] - norm / 2
print(midpoint.shape)

(16896, 3)
(16896, 3)
(16896, 3)
(16896, 3, 3)
(16896, 3)


In [5]:
xyz = midpoint.astype('int32')
np.all(cube_world[xyz[:, 0], xyz[:, 1], xyz[:, 2]] == 1)

True

In [6]:
import os
import cv2
import pickle

In [99]:
block_path = "MinecraftTextures/block/"

MAX_NUM_MINECRAFT_TEXTURES = len(list(os.listdir(block_path)))

filter_list = []

texture_width = 16
texture = np.zeros((MAX_NUM_MINECRAFT_TEXTURES * texture_width, texture_width, 3))
block_to_texture = {}

cur_coord = 0
for file in os.listdir(block_path):
    if not file.endswith(".png"):
        continue
    
    t_offset = cur_coord
    y = t_offset/(MAX_NUM_MINECRAFT_TEXTURES * texture_width)
    block_to_texture[file] = [[0, 1 - y], [1, 1 - y], [1, 1 - (y + 1/MAX_NUM_MINECRAFT_TEXTURES)]]
#     block_to_texture[file] = [[0, 0], [1, 0], [1, 1/MAX_NUM_MINECRAFT_TEXTURES]]

    
    image = cv2.imread(f"{block_path}{file}")
    texture[cur_coord:cur_coord + texture_width,:] = image
    cur_coord += texture_width
    
with open("block_to_texture.pkl", "wb") as f:
    pickle.dump(block_to_texture, f)

cv2.imwrite("all_mc_textures.png", texture)

True

In [100]:
with open("objs/p3d/results/voxel_to_block_mesh_1", "rb") as f:
    voxel_to_block = pickle.load(f)

In [101]:
vt_lines = []
face_lines = []
ft_idx = 1
for f, voxel in zip(obj["f"], xyz):
    block = voxel_to_block[tuple(voxel)]
    
    vt = block_to_texture[block]
    vt_lines.append("\n".join(["vt " + x for x in map(lambda v: " ".join([str(y) for y in v]), vt)]))
    
    
    line = "f "
    for i in range(3):
        line += f"{f[i]}/{ft_idx} "
        ft_idx += 1
    
    face_lines.append(line)

In [102]:
with open("FORMAT.mtl", "r") as f:
    mtl_text = f.read().replace("MESH", "mesh_1")

In [103]:
with open("mesh_1.mtl", "w") as f:
    f.write(mtl_text)

In [104]:
with open("textured_mesh.obj", "w") as f:
#     mtllib mesh_0.mtl
    contents = "mtllib mesh_1.mtl\n"
    contents += "\n".join(["v " + x for x in map(lambda v: " ".join([str(y) for y in v]), obj["v"])])
    contents += "\n"
    contents += "\n".join(vt_lines)
    contents += "\n"
    contents += "usemtl mesh_1" + "\n"
    contents += "\n".join(face_lines)
    f.write(contents)